In [18]:
# ============================================
# 48音の物理的特徴量（強化版）一括抽出スクリプト
# ============================================

from pathlib import Path
import re
import unicodedata

import numpy as np
import pandas as pd
import librosa
import librosa.display
from tqdm import tqdm
from scipy.signal import hilbert

# ------------------------------------------------
# 0. パラメータ設定
# ------------------------------------------------
ROOT_DIR = Path("/Users/shunsuke/EEG_48sounds")
DERIV_DIR = ROOT_DIR / "derivatives"

OUT_DIR = DERIV_DIR / "sound_features"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ★ここが最重要：ROOT基準で音刺激リストを指す
AUDIO_DIR = ROOT_DIR / "音刺激リスト"

# サブフォルダ含めてwav探索（.WAVも拾う）
wav_files = [p for p in AUDIO_DIR.rglob("*") if p.is_file() and p.suffix.lower() == ".wav"]
wav_files = sorted(wav_files)

if len(wav_files) == 0:
    raise FileNotFoundError(f"wavが見つかりません: {AUDIO_DIR}")


SR_TARGET   = 22050
N_FFT       = 2048
HOP_LENGTH  = 512
FMIN        = 80.0
FMAX        = 1000.0
N_MFCC      = 13

SR_ENV      = 200.0



# ------------------------------------------------
# 1. ファイル名から番号(sound_no)を取る
# ------------------------------------------------
def get_sound_number_from_name(name: str) -> int:
    normalized = unicodedata.normalize("NFKC", name)
    m = re.match(r"^(\d+)\s*[:：]", normalized)
    if not m:
        raise ValueError(f"番号が読み取れませんでした: {name}")
    return int(m.group(1))

def strip_number_prefix(name: str) -> str:
    """'12：xxx.wav' の先頭番号を落として 'xxx.wav' にする。"""
    s = unicodedata.normalize("NFKC", str(name)).strip()
    s = re.sub(r"^\s*\d+\s*[:：]\s*", "", s)
    return s

def normalize_filename(name: str) -> str:
    """主観評価側と同じ正規化：番号接頭辞・-9dBA等を除去してキー化。"""
    s = strip_number_prefix(name)
    s = re.sub(r"(_-?\d+dBA.*)\.wav$", ".wav", s, flags=re.IGNORECASE)
    return s


# ------------------------------------------------
# 2. STFT から周波数バンドのエネルギー[dB]
# ------------------------------------------------
def band_energy_db(S, freqs, fmin, fmax):
    idx = np.logical_and(freqs >= fmin, freqs < fmax)
    if not np.any(idx):
        return np.nan
    band_power = np.mean(S[idx, :])
    band_power_db = 10 * np.log10(band_power + 1e-12)
    return float(band_power_db)


# ------------------------------------------------
# 3. 1ファイル分の特徴量抽出
# ------------------------------------------------
def extract_features(path: Path) -> dict:
    # --- 読み込み ---
    y, sr = librosa.load(path, sr=SR_TARGET, mono=True)
    duration_s = librosa.get_duration(y=y, sr=sr)

    # --- STFT・基本量 ---
    rms = librosa.feature.rms(y=y, frame_length=N_FFT, hop_length=HOP_LENGTH)[0]
    S = np.abs(librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH))**2  # power
    freqs = librosa.fft_frequencies(sr=sr, n_fft=N_FFT)

    # スペクトル形状（フレーム単位）
    spec_centroid = librosa.feature.spectral_centroid(S=S, sr=sr)[0]
    spec_bw       = librosa.feature.spectral_bandwidth(S=S, sr=sr)[0]
    rolloff85     = librosa.feature.spectral_rolloff(S=S, sr=sr, roll_percent=0.85)[0]
    rolloff95     = librosa.feature.spectral_rolloff(S=S, sr=sr, roll_percent=0.95)[0]
    flatness      = librosa.feature.spectral_flatness(S=S)[0]
    zcr           = librosa.feature.zero_crossing_rate(
                        y, frame_length=N_FFT, hop_length=HOP_LENGTH
                    )[0]

    # スペクトル変化
    spec_flux = librosa.onset.onset_strength(y=y, sr=sr)

    # --- 時間構造：エンベロープ・アタック ---
    analytic = hilbert(y)
    env = np.abs(analytic)
    if env.max() > 0:
        env_norm   = env / env.max()
        idx_attack = np.argmax(env_norm > 0.9)
        attack_time = idx_attack / sr
        env_var    = float(np.var(env_norm))
    else:
        attack_time = np.nan
        env_var     = np.nan

    # オンセット密度
    onset_env   = librosa.onset.onset_strength(y=y, sr=sr)
    onset_peaks = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)
    onset_density = len(onset_peaks) / duration_s if duration_s > 0 else np.nan

    # 時間的重心 & ダイナミックレンジ
    frame_times = librosa.frames_to_time(
        np.arange(len(rms)), sr=sr, hop_length=HOP_LENGTH
    )
    energy = rms**2
    temporal_centroid = float(
        np.sum(frame_times * energy) / (np.sum(energy) + 1e-12)
    )
    rms_db = librosa.amplitude_to_db(rms, ref=np.max)
    dynamic_range_db = float(
        np.percentile(rms_db, 95) - np.percentile(rms_db, 5)
    )

    # --- ピッチ ---
    try:
        f0 = librosa.yin(y, fmin=FMIN, fmax=FMAX, sr=sr)
        f0_valid = f0[f0 > 0]
        if len(f0_valid) > 0:
            f0_mean      = float(np.mean(f0_valid))
            f0_std       = float(np.std(f0_valid))
            voiced_ratio = float(len(f0_valid) / len(f0))
        else:
            f0_mean      = np.nan
            f0_std       = np.nan
            voiced_ratio = 0.0
    except Exception:
        f0_mean      = np.nan
        f0_std       = np.nan
        voiced_ratio = 0.0

    # --- ハーモニック / パーカッシブ ---
    y_harm, y_perc = librosa.effects.hpss(y)
    harm_energy = float(np.sum(y_harm**2))
    perc_energy = float(np.sum(y_perc**2))
    total_energy = harm_energy + perc_energy + 1e-12
    harmonic_energy_ratio   = harm_energy / total_energy
    percussive_energy_ratio = perc_energy / total_energy

    # --- 周波数帯エネルギー ---
    band_low_db  = band_energy_db(S, freqs, 0,    250)
    band_mid_db  = band_energy_db(S, freqs, 250,  2000)
    band_high_db = band_energy_db(S, freqs, 2000, 8000)
    high_low_ratio = (
        band_high_db - band_low_db
        if not (np.isnan(band_high_db) or np.isnan(band_low_db))
        else np.nan
    )

    # --- スペクトルの高次統計（平均スペクトルから） ---
    spec_mean = np.mean(S, axis=1)     # freq 軸の平均パワー
    spec_prob = spec_mean / (np.sum(spec_mean) + 1e-12)

    # 有効帯域（人間の聴覚帯域にざっくり制限）
    valid = np.logical_and(freqs >= 20, freqs <= 20000)
    f_valid = freqs[valid]
    p_valid = spec_prob[valid]

    # 正規化し直し
    p_valid = p_valid / (np.sum(p_valid) + 1e-12)

    mu = np.sum(f_valid * p_valid)
    spread = np.sqrt(np.sum(((f_valid - mu)**2) * p_valid))
    skew = np.sum(((f_valid - mu)**3) * p_valid) / (spread**3 + 1e-12)
    kurt = np.sum(((f_valid - mu)**4) * p_valid) / (spread**4 + 1e-12)
    entropy = -np.sum(p_valid * np.log2(p_valid + 1e-12)) / np.log2(len(p_valid))

    # スペクトル傾き（dB/kHz）
    log_spec = 10 * np.log10(spec_mean + 1e-12)
    # 100〜8000 Hz に制限して回帰
    mask = np.logical_and(freqs >= 100, freqs <= 8000)
    f_reg = freqs[mask] / 1000.0  # kHz
    y_reg = log_spec[mask]
    A = np.vstack([f_reg, np.ones_like(f_reg)]).T
    slope, intercept = np.linalg.lstsq(A, y_reg, rcond=None)[0]
    spec_slope_db_per_khz = float(slope)

    # --- MFCC ---
    mfcc = librosa.feature.mfcc(
        S=librosa.power_to_db(S),
        sr=sr,
        n_mfcc=N_MFCC
    )

    # --- 振幅変調スペクトル ---
    # envelope を低サンプリングレートにリサンプル
    env_ds = librosa.resample(env, orig_sr=sr, target_sr=SR_ENV)
    N_env = len(env_ds)
    env_ds = env_ds - np.mean(env_ds)
    E = np.fft.rfft(env_ds * np.hanning(N_env))
    freqs_env = np.fft.rfftfreq(N_env, d=1.0 / SR_ENV)
    mod_power = np.abs(E)**2
    total_mod_power = np.sum(mod_power) + 1e-12

    def am_band_ratio(fmin, fmax):
        idx = np.logical_and(freqs_env >= fmin, freqs_env < fmax)
        if not np.any(idx):
            return 0.0
        return float(np.sum(mod_power[idx]) / total_mod_power)

    am_band_0_2   = am_band_ratio(0.0, 2.0)
    am_band_2_8   = am_band_ratio(2.0, 8.0)
    am_band_8_20  = am_band_ratio(8.0, 20.0)
    am_centroid_hz = float(np.sum(freqs_env * mod_power) / total_mod_power)

    # --- スカラー特徴量まとめ ---
    feat = {
        "file_name": path.name,

        # 時間・レベル
        "duration_s":         duration_s,
        "rms_mean":           float(np.mean(rms)),
        "rms_std":            float(np.std(rms)),
        "rms_max":            float(np.max(rms)),
        "peak_amp":           float(np.max(np.abs(y))),
        "crest_factor_db":    float(
            20 * np.log10((np.max(np.abs(y)) + 1e-12) /
                          (np.mean(rms) + 1e-12))
        ),
        "temporal_centroid_s": temporal_centroid,
        "dynamic_range_db":    dynamic_range_db,

        # スペクトル形状
        "spec_centroid_mean": float(np.mean(spec_centroid)),
        "spec_centroid_std":  float(np.std(spec_centroid)),
        "spec_bw_mean":       float(np.mean(spec_bw)),
        "rolloff85_mean":     float(np.mean(rolloff85)),
        "rolloff95_mean":     float(np.mean(rolloff95)),
        "flatness_mean":      float(np.mean(flatness)),
        "zcr_mean":           float(np.mean(zcr)),

        # 高次スペクトル統計
        "spec_entropy":       float(entropy),
        "spec_spread":        float(spread),
        "spec_skewness":      float(skew),
        "spec_kurtosis":      float(kurt),
        "spec_slope_db_per_khz": spec_slope_db_per_khz,

        # 時間変化
        "spec_flux_mean":     float(np.mean(spec_flux)),
        "attack_time_s":      float(attack_time),
        "env_var":            env_var,
        "onset_density":      float(onset_density),

        # ピッチ関連
        "f0_mean":            f0_mean,
        "f0_std":             f0_std,
        "voiced_ratio":       voiced_ratio,

        # ハーモニック / パーカッシブ
        "harmonic_energy_ratio":   harmonic_energy_ratio,
        "percussive_energy_ratio": percussive_energy_ratio,

        # バンドエネルギー
        "band_low_db":        band_low_db,
        "band_mid_db":        band_mid_db,
        "band_high_db":       band_high_db,
        "high_low_ratio_db":  high_low_ratio,

        # AM 特徴
        "am_band_0_2_ratio":  am_band_0_2,
        "am_band_2_8_ratio":  am_band_2_8,
        "am_band_8_20_ratio": am_band_8_20,
        "am_centroid_hz":     am_centroid_hz,
    }

    # MFCC1〜N_MFCC の平均・分散
    for i in range(mfcc.shape[0]):
        feat[f"mfcc{i+1}_mean"] = float(np.mean(mfcc[i]))
        feat[f"mfcc{i+1}_std"]  = float(np.std(mfcc[i]))

    return feat


# ------------------------------------------------
# 4. wav 一覧取得 & 番号付与
# ------------------------------------------------
print(f"検出した wav ファイル数: {len(wav_files)}")


pairs = []
bad = []

for p in wav_files:
    try:
        n = get_sound_number_from_name(p.name)
        pairs.append((n, p))
    except Exception as e:
        bad.append((p.name, str(e)))

if bad:
    pd.DataFrame(bad, columns=["file_name", "reason"]).to_csv(
        OUT_DIR / "audit_bad_filename_for_number.csv",
        index=False, encoding="utf-8-sig"
    )
    print("⚠️ 番号抽出できないwavがありました →", OUT_DIR / "audit_bad_filename_for_number.csv")
    # 番号方式で進めるならここで止める
    raise ValueError("番号抽出できないwavがあります。監査CSVを確認してください。")

pairs.sort(key=lambda x: x[0])

print(f"pairs: {len(pairs)}")
print("番号 → ファイル名（先頭10件）:")
for n, p in pairs[:10]:
    print(f"{n:2d} -> {p.name}")

if len(pairs) != 48:
    print("⚠️ 注意: 48音ではありません。現状:", len(pairs))



# ------------------------------------------------
# 5. 48音すべての特徴量を計算
# ------------------------------------------------
rows = []
for n, p in tqdm(pairs):
    feat = extract_features(p)

    # 統合用キー（★最重要）
    feat["number"] = int(n)
    feat["sound_id"] = f"S{n:02d}"
    feat["FileName"] = strip_number_prefix(p.name)
    feat["FileName_norm"] = normalize_filename(p.name)
    feat["FileName_norm_unique"] = f"{n:02d}__{feat['FileName_norm']}"
    feat["sound_label"] = f"{n:02d}:{feat['FileName_norm']}"        # 例: "20:motorcycle.wav"
    feat["audio_relpath"] = str(p.relative_to(AUDIO_DIR))           # 例: "20：motorcycle_-9dBA.wav"


    rows.append(feat)

df = pd.DataFrame(rows)

id_cols = [
    "number","sound_id",
    "sound_label",
    "FileName","FileName_norm","FileName_norm_unique",
    "file_name",
    "audio_relpath",
]

other_cols = [c for c in df.columns if c not in id_cols]
df = df[id_cols + other_cols].copy()

# 監査：48音・キー一意性
df = df.sort_values("number").reset_index(drop=True)

# number は絶対に一意（ここだけは落とす）
if df["number"].nunique() != len(df):
    dup = df[df.duplicated("number", keep=False)].sort_values("number")
    dup.to_csv(OUT_DIR / "audit_duplicate_number_rows.csv", index=False, encoding="utf-8-sig")
    raise ValueError("number が重複しています。audit_duplicate_number_rows.csv を確認してください。")

# FileName_norm は重複OK（WARN＋監査CSV）
dup_mask = df.duplicated("FileName_norm", keep=False)
if dup_mask.any():
    dup = df.loc[dup_mask, ["number","file_name","FileName","FileName_norm"]]\
            .sort_values(["FileName_norm","number"])
    dup.to_csv(OUT_DIR / "audit_duplicate_FileName_norm_rows.csv", index=False, encoding="utf-8-sig")
    print("⚠️ [WARN] FileName_norm が重複（number を主キーとして処理継続）")

expected = set(range(1, 49))
got = set(df["number"].tolist())
missing = sorted(expected - got)
extra = sorted(got - expected)
if missing or extra:
    raise ValueError(f"number 欠番/過剰があります missing={missing} extra={extra}")



if len(df) != 48:
    raise ValueError(f"行数が48ではありません（{len(df)}行）。")


# ------------------------------------------------
# 6. CSV に保存
# ------------------------------------------------
out_path = OUT_DIR / "sound_features_48.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")


print("\n==== 完了 ====")
print(f"出力ファイル: {out_path}")
print(f"データ形状: {df.shape[0]} 行 × {df.shape[1]} 列")
display(df.head())



検出した wav ファイル数: 48
pairs: 48
番号 → ファイル名（先頭10件）:
 1 -> 1：flute_-9dBA_final.wav
 2 -> 2：yawn_-9dBA.wav
 3 -> 3：winding_clock_-9dBA.wav
 4 -> 4：drum_-9dBA.wav
 5 -> 5：roller_coaster_-9dBA_final.wav
 6 -> ６：(computer)_mouse_-9dBA.wav
 7 -> 7：trumpet_-9dBA_final.wav
 8 -> 8：cricket_-9dBA.wav
 9 -> 9：cheering_crowd_-9dBA.wav
10 -> 10：horse_-9dBA.wav


100%|██████████| 48/48 [00:08<00:00,  5.86it/s]

⚠️ [WARN] FileName_norm が重複（number を主キーとして処理継続）
FileName_norm
motorcycle.wav    [20, 29]

==== 完了 ====
出力ファイル: /Users/shunsuke/EEG_48sounds/derivatives/sound_features/sound_features_48.csv
データ形状: 48 行 × 71 列


,number,sound_id,sound_label,FileName,FileName_norm,FileName_norm_unique,file_name,audio_relpath,duration_s,rms_mean,...,mfcc9_mean,mfcc9_std,mfcc10_mean,mfcc10_std,mfcc11_mean,mfcc11_std,mfcc12_mean,mfcc12_std,mfcc13_mean,mfcc13_std
0,1,S01,01:flute.wav,flute_-9dBA_final.wav,flute.wav,01__flute.wav,1：flute_-9dBA_final.wav,1：flute_-9dBA_final.wav,5.000000,0.066950,...,-8.849242,23.084324,12.059900,14.860441,-11.425395,20.371939,1.776582,13.677643,-12.830481,17.981844
1,2,S02,02:yawn.wav,yawn_-9dBA.wav,yawn.wav,02__yawn.wav,2：yawn_-9dBA.wav,2：yawn_-9dBA.wav,4.084036,0.010090,...,-6.891694,18.213989,11.111100,20.253798,-28.253290,33.115227,7.111513,22.100752,-23.539501,28.667854
2,3,S03,03:winding_clock.wav,winding_clock_-9dBA.wav,winding_clock.wav,03__winding_clock.wav,3：winding_clock_-9dBA.wav,3：winding_clock_-9dBA.wav,4.868934,0.005338,...,-49.273891,31.536839,45.674500,20.663122,-37.494339,20.546431,32.762329,14.867008,-49.713551,26.487375
3,4,S04,04:drum.wav,drum_-9dBA.wav,drum.wav,04__drum.wav,4：drum_-9dBA.wav,4：drum_-9dBA.wav,4.671565,0.029745,...,1.062432,22.807062,15.333307,17.183823,-4.530825,17.201454,15.843509,14.259997,0.780155,16.097494
4,5,S05,05:roller_coaster.wav,roller_coaster_-9dBA_final.wav,roller_coaster.wav,05__roller_coaster.wav,5：roller_coaster_-9dBA_final.wav,5：roller_coaster_-9dBA_final.wav,5.000000,0.016387,...,-8.441719,12.650905,15.268452,12.806000,-21.787670,22.921701,16.964586,13.584641,-1.672230,11.729448
